<a href="https://colab.research.google.com/github/mancher07/Stable-Diffusion-3/blob/main/SD3_jupyter_Master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content
!git clone -b master https://github.com/camenduru/ComfyUI /content/MasterUI
%cd /content/MasterUI

!pip install torch numpy pillow
!pip install -q torchsde einops diffusers accelerate xformers==0.0.26.post1
!apt -y install -qq aria2

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/adamo1139/stable-diffusion-3-medium-ungated/resolve/main/sd3_medium_incl_clips_t5xxlfp8.safetensors -d /content/MasterUI/model -o sd3_medium_incl_clips_t5xxlfp8.safetensors

import torch
import random
import node_helpers
from comfy.sd import load_checkpoint_guess_config
import nodes
import numpy as np
from PIL import Image
from google.colab import files

with torch.inference_mode():
    model_patcher, clip, vae, clipvision = load_checkpoint_guess_config("/content/MasterUI/model/sd3_medium_incl_clips_t5xxlfp8.safetensors", output_vae=True, output_clip=True, embedding_directory=None)

/content
Cloning into '/content/MasterUI'...
remote: Enumerating objects: 11621, done.
remote: Total 11621 (delta 0), reused 0 (delta 0), pack-reused 11621
Receiving objects: 100% (11621/11621), 5.72 MiB | 13.98 MiB/s, done.
Resolving deltas: 100% (7886/7886), done.
/content/MasterUI
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia

In [8]:
# Simular las funciones ausentes
class comfy:
    class sample:
        @staticmethod
        def fix_empty_latent_channels(model, latent_image):
            return latent_image

        @staticmethod
        def prepare_noise(latent_image, seed, batch_inds):
            torch.manual_seed(seed)
            return torch.randn_like(latent_image)

    class utils:
        PROGRESS_BAR_ENABLED = True

class latent_preview:
    @staticmethod
    def prepare_callback(model, steps):
        return None

# Función para cargar y procesar la imagen de estilo
def load_style_image():
    uploaded = files.upload()
    for filename in uploaded.keys():
        image = Image.open(filename)
        return image

class KSampler:
    @classmethod
    def INPUT_TYPES(cls):
        return {"required": {
                    "model": ("MODEL",),
                    "seed": ("INT", {"default": 0, "min": 0, "max": 0xffffffffffffffff}),
                    "steps": ("INT", {"default": 20, "min": 1, "max": 10000}),
                    "cfg": ("FLOAT", {"default": 8.0, "min": 0.0, "max": 100.0, "step": 0.1, "round": 0.01}),
                    "sampler_name": ("SAMPLER_NAME",),
                    "scheduler": ("SCHEDULER",),
                    "positive": ("CONDITIONING",),
                    "negative": ("CONDITIONING",),
                    "latent_image": ("LATENT",),
                    "denoise": ("FLOAT", {"default": 1.0, "min": 0.0, "max": 1.0, "step": 0.01})
                    }
                }

    RETURN_TYPES = ("LATENT",)
    FUNCTION = "sample"
    CATEGORY = "sampling"

    def sample(self, model, seed, steps, cfg, sampler_name, scheduler, positive, negative, latent_image, denoise=1.0):
        return common_ksampler(model, seed, steps, cfg, sampler_name, scheduler, positive, negative, latent_image, denoise=denoise)


class ImportStyleImage:
    @classmethod
    def INPUT_TYPES(cls):
        return {"required": {
                    "style_image_path": ("STRING",),
                    "prompt": ("STRING",)
                    }
                }

    RETURN_TYPES = ("CONDITIONING", "STRING")
    FUNCTION = "import_style"
    CATEGORY = "image"

    def import_style(self, style_image_path, prompt):
        style_image = Image.open(style_image_path)
        style_image = np.array(style_image)
        conditioning_data = {"style_image": style_image, "prompt": prompt}
        return conditioning_data, json.dumps(conditioning_data)


class SaveImage:
    def __init__(self):
        self.output_dir = "./output"
        self.prefix_append = ""
        self.compress_level = 4

    @classmethod
    def INPUT_TYPES(cls):
        return {"required":
                    {"images": ("IMAGE",),
                     "filename_prefix": ("STRING", {"default": "ComfyUI"})}
                }

    RETURN_TYPES = ()
    FUNCTION = "save_images"
    OUTPUT_NODE = True
    CATEGORY = "image"

    def save_images(self, images, filename_prefix="ComfyUI"):
        for image in images:
            img = Image.fromarray(np.clip(image.cpu().numpy() * 255, 0, 255).astype(np.uint8))
            img.save(f"{self.output_dir}/{filename_prefix}.png", compress_level=self.compress_level)


def common_ksampler(model, seed, steps, cfg, sampler_name, scheduler, positive, negative, latent, denoise=1.0, disable_noise=False, start_step=None, last_step=None, force_full_denoise=False):
    latent_image = latent["samples"]

    if disable_noise:
        noise = torch.zeros(latent_image.size(), dtype=latent_image.dtype, layout=latent_image.layout, device=latent_image.device)
    else:
        noise = torch.randn_like(latent_image)

    samples = latent_image + noise  # Este es un simple ejemplo, debería reemplazarse con la lógica de muestra real

    out = latent.copy()
    out["samples"] = samples
    return out

# Ejemplo de uso
with torch.inference_mode():
    # Cargar y procesar la imagen de estilo
    style_image = load_style_image()
    style_image = style_image.resize((1024, 1024))
    style_tensor = torch.from_numpy(np.array(style_image)).unsqueeze(0).permute(0, 3, 1, 2).float() / 255.0

    latent = {"samples": torch.ones([1, 16, 1024 // 8, 1024 // 8]) * 0.0609}
    prompt = "a female character with long, flowing hair that appears to be made of ethereal, swirling patterns resembling the Northern Lights or Aurora Borealis. The background is dominated by deep blues and purples, creating a mysterious and dramatic atmosphere. The character's face is serene, with pale skin and striking features. She wears a dark-colored outfit with subtle patterns. The overall style of the artwork is reminiscent of fantasy or supernatural genres"
    cond = [[torch.randn([1, 768]), {"pooled_output": torch.randn([1, 768])}]]

    negative_prompt = "bad quality, poor quality, doll, disfigured, jpg, toy, bad anatomy, missing limbs, missing fingers, 3d, cgi"
    n_cond = [[torch.randn([1, 768]), {"pooled_output": torch.randn([1, 768])}]]

    seed = 1
    if seed == 0:
        seed = random.randint(0, 18446744073709551615)
    print(seed)

    # Utilizar la imagen de estilo en el modelo de generación
    sample = common_ksampler(
        model=None,  # Reemplazar con el modelo adecuado
        seed=seed,
        steps=28,
        cfg=4.5,
        sampler_name="dpmpp_2m",
        scheduler="sgm_uniform",
        positive=cond,
        negative=n_cond,
        latent=latent,
        denoise=1
    )

    sample = sample["samples"].to(torch.float16)

    # Simulación de decodificación, reemplazar con la decodificación adecuada
    decoded = sample.squeeze(0).permute(1, 2, 0).cpu().numpy()

    # Asegurarse de que el tensor tiene la forma adecuada para la conversión a imagen PIL
    if decoded.shape[-1] == 16:  # Si la última dimensión es 16, asumimos que es un tensor de características
        decoded = np.mean(decoded, axis=-1)  # Promediar las características para obtener una imagen de una sola canal

    Image.fromarray((decoded * 255).astype(np.uint8)).show()

Saving abomination.png to abomination (4).png
1
